## AML 2304 Lab 3 - Group E

In [1]:
import os
import pandas as pd
import email
import re
import nltk
import contractions
from email import policy
from email.header import decode_header
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.metrics import edit_distance, jaccard_distance
from collections import Counter

nltk.download('wordnet')
pd.set_option('max_colwidth', 100)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aurad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Read file contents from Dataset main folder. Extract the `SUBJECT`, `TO`, `FROM` and `EMAIL BODY` from the email, then add the data in dataframe with the correct label: `spam` or `ham`

In [2]:
def get_email_body(msg):
    email_body = ""

    if "multipart" in msg.get_content_type():

        for part in msg.get_payload(decode=False):
            if not isinstance(part, str):
                email_part = str(part.get_payload(decode=False)).strip()
                email_body = email_body + email_part
    else:
        email_body = str(msg.get_payload(decode=False)).strip()

    return email_body


try:
    data_raw_df = pd.DataFrame(columns=['subject', 'to', 'from', 'email_body', 'label'])

    labels = ['ham', 'spam']
    for idx in range(len(labels)):

        spam_dir = "Dataset/" + labels[idx] + "/"
        # Read all files from the dataset folder
        for file in os.listdir(spam_dir)[:]:
            with open(spam_dir + file, "r", encoding="latin-1") as f:
                # Parse the email messages
                msg = email.message_from_string(f.read(), policy=policy.default)

                subject = str(decode_header(msg.get('subject'))[0][0]) if msg.get('subject') is not None else None
                email_body = get_email_body(msg)

                # Add label to your record: 0 = ham, 1 = spam
                new_data = [subject, msg.get('to'), msg.get('from'), email_body, idx]
                data_raw_df.loc[len(data_raw_df)] = new_data

    display(data_raw_df.head(5))

except Exception as err:
    print(f"Error: {err} - filename: {file}")

,subject,to,from,email_body,label
0,Re: New Sequences Window,Chris Garrigues <cwg-dated-1030314468.7c7c85@DeepEddy.Com>,Robert Elz <kre@munnari.OZ.AU>,"Date: Tue, 20 Aug 2002 17:27:47 -0500\n From: Chris Garrigues <cwg-exmh@DeepEdd...",0
1,Re: New Sequences Window,Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>,Robert Elz <kre@munnari.OZ.AU>,"Date: Wed, 21 Aug 2002 10:54:46 -0500\n From: Chris Garrigues <cwg-dated-103037...",0
2,Personal Finance: Resolutions You Can Keep,mkettler@home.com,The Motley Fool <Fool@motleyfool.com>,======================== THE MOTLEY FOOL ========================\n PERSO...,0
3,Re: New Sequences Window,Valdis.Kletnieks@vt.edu,Chris Garrigues <cwg-exmh@DeepEddy.Com>,"> From: Valdis.Kletnieks@vt.edu\n> Date: Wed, 21 Aug 2002 02:36:56 -0400\n>\n> --==_Exmh_77858...",0
4,[zzzzteana] RE: Alexander,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoogroups.com>",Steve Burt <Steve_Burt@cursor-system.com>,"Martin A posted:\nTassos Papadopoulos, the Greek sculptor behind the plan, judged that the\n lim...",0


## Exploratory Data Analysis (incl. cleaning the data)

In [9]:
# Check if there are null values
print(data_raw_df.isnull().sum())

subject       0
to            0
from          0
email_body    0
label         0
dtype: int64


In [10]:
# Replace all None data with empty string
data_raw_df = data_raw_df.fillna('')
data_raw_df.isnull().sum()

subject       0
to            0
from          0
email_body    0
label         0
dtype: int64

In [11]:
# Check the total records for each column grouped by label
data_raw_df.groupby('label').agg('count')

,subject,to,from,email_body
label,,,,
0,6701,6701,6701,6701
1,1896,1896,1896,1896


In [ ]:
# This variable will contain the cleaned data
data_cleaned_df = data_raw_df.copy()
# Convert to lower case characters
data_cleaned_df = data_cleaned_df.applymap(lambda x: x.lower() if type(x) == str else x)

data_cleaned_df.head(5)

,subject,to,from,email_body,label
0,re: new sequences window,Chris Garrigues <cwg-dated-1030314468.7c7c85@DeepEddy.Com>,Robert Elz <kre@munnari.OZ.AU>,"date: tue, 20 aug 2002 17:27:47 -0500\n from: chris garrigues <cwg-exmh@deepedd...",0
1,re: new sequences window,Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>,Robert Elz <kre@munnari.OZ.AU>,"date: wed, 21 aug 2002 10:54:46 -0500\n from: chris garrigues <cwg-dated-103037...",0
2,personal finance: resolutions you can keep,mkettler@home.com,The Motley Fool <Fool@motleyfool.com>,======================== the motley fool ========================\n perso...,0
3,re: new sequences window,Valdis.Kletnieks@vt.edu,Chris Garrigues <cwg-exmh@DeepEddy.Com>,"> from: valdis.kletnieks@vt.edu\n> date: wed, 21 aug 2002 02:36:56 -0400\n>\n> --==_exmh_77858...",0
4,[zzzzteana] re: alexander,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoogroups.com>",Steve Burt <Steve_Burt@cursor-system.com>,"martin a posted:\ntassos papadopoulos, the greek sculptor behind the plan, judged that the\n lim...",0


Extract mail server and domain name from `TO` and `FROM`

In [ ]:
def extract_mail_server_and_domain(email):
    pattern = r'@([\w.]+\.[a-zA-Z]{2,})'
    match = re.search(pattern, email)
    return match.group(1) if match else None
    

data_cleaned_df['to'] = data_cleaned_df['to'].apply(extract_mail_server_and_domain)
data_cleaned_df['from'] = data_cleaned_df['from'].apply(extract_mail_server_and_domain)
data_cleaned_df[['to', 'from']].head(5)

,to,from
0,DeepEddy.Com,munnari.OZ.AU
1,DeepEddy.Com,munnari.OZ.AU
2,home.com,motleyfool.com
3,vt.edu,DeepEddy.Com
4,yahoogroups.com,None


For `SUBJECT` and `EMAIL_BODY`, the following will be applied to clean the data
- Remove email addresses and URLs
- Remove HTML tags and scripting variable names
- Replace all whitespaces with space
- Handling contractions
- Remove special characters and numbers
- Tokenization, stopwords removal and lemmatization
- Handling typos and mispellings (?)
- Remove words that are not in dictionary (?)

In [ ]:
pd.set_option('max_colwidth', None)


# Remove email addresses
pattern_email = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
data_cleaned_df[['subject', 'email_body']] = data_cleaned_df[['subject', 'email_body']].applymap(lambda x : re.sub(pattern_email, '', x))

# Remove URLs
pattern_url = r'https?://(?:www\.)?[\w\.-]+(?:\.[a-zA-Z]{2,})+(?:/[-\w\.,/]*)*(?:\?[\w\%&=]*)?'
data_cleaned_df[['subject', 'email_body']] = data_cleaned_df[['subject', 'email_body']].applymap(lambda x : re.sub(pattern_url, '', x))

# Handle contractions
data_cleaned_df[['subject', 'email_body']] = data_cleaned_df[['subject', 'email_body']].applymap(lambda x : contractions.fix(x))

# Remove HTML tags
pattern_html = r'<[^>]+>'
data_cleaned_df[['subject', 'email_body']] = data_cleaned_df[['subject', 'email_body']].applymap(lambda x : re.sub(pattern_html, '', x))
data_cleaned_df[['subject', 'email_body']] = data_cleaned_df[['subject', 'email_body']].applymap(lambda x : x.replace('&nbsp;', ' '))

# Remove scripting variable names, e.g. $msg
pattern_scripting = r'\$\w+\s*'
data_cleaned_df[['subject', 'email_body']] = data_cleaned_df[['subject', 'email_body']].applymap(lambda x : re.sub(pattern_scripting, '', x))


# Remove all special characters and numbers
pattern_special_num = r'(?<![a-zA-Z])-(?![a-zA-Z])|[^a-zA-Z\s-]+'
data_cleaned_df[['subject', 'email_body']] = data_cleaned_df[['subject', 'email_body']].applymap(lambda x : re.sub(pattern_special_num, ' ', x))

# Replace all whitespaces (new lines, tabs, etc.) to space
pattern_whitespace = r'\s+'
data_cleaned_df[['subject', 'email_body']] = data_cleaned_df[['subject', 'email_body']].applymap(lambda x : re.sub(pattern_whitespace, ' ', x))

display(data_cleaned_df.head(2))
display(data_cleaned_df.tail(2))

subject            to           from  \
0  re new sequences window  DeepEddy.Com  munnari.OZ.AU   
1  re new sequences window  DeepEddy.Com  munnari.OZ.AU   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

subject           to          from  \
8595  cannabis difference  webnote.net  dialix.oz.au   
8596   ilug wilson kamela     linux.ie  netscape.net   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
# Tokenize the text and remove stopwords
stop_words = set(stopwords.words('english'))
data_cleaned_df[['subject', 'email_body']] = data_cleaned_df[['subject', 'email_body']].applymap(lambda x : [word for word in word_tokenize(x) if word not in stop_words])

display(data_cleaned_df.head(2))
display(data_cleaned_df.tail(2))

,subject,to,from,email_body,label
0,"[new, sequences, window]",DeepEddy.Com,munnari.OZ.AU,"[date, tue, aug, chris, garrigues, message-id, hoping, people, additional, sequences, notice, purely, cosmetic, changes, well, first, exmh, latest, one, changes, starts, get, read, flist, totalcount, unseen, element, array, executing, totalcount, unseen-sequence, flaginner, spool, iconspool, labelup, else, flaginner, icondown, labeldown, procedure, flag, msgseen, line, invoked, within, flag, msgseen, procedure, msgseen, line, invoked, within, msgseen, procedure, msgshow, line, invoked, within, msgshow, procedure, msgchange, line, invoked, within, msgchange, show, invoked, within, time, list, msgchange, procedure, msg, change, line, invoked, within, msg, change, id, procedure, msg, show, line, invoked, within, msg, show, cur, eval, body, line, invoked, within, eval, procedure, folderchange, line, invoked, within, ...]",0
1,"[new, sequences, window]",DeepEddy.Com,munnari.OZ.AU,"[date, wed, aug, chris, garrigues, message-id, reproduce, error, repeatable, like, every, time, without, fail, debug, log, pick, happening, pick, exec, pick, inbox, -list, -lbrace, -lbrace, -subject, ftp, -rbrace, -rbrace, -sequence, mercury, exec, pick, inbox, -list, -lbrace, -lbrace, -subject, ftp, -rbrace, -rbrace, -sequence, mercury, ftoc, pickmsgs, hit, marking, hits, tkerror, syntax, error, expression, int, note, run, pick, command, hand, delta, pick, inbox, -list, -lbrace, -lbrace, -subject, ftp, -rbrace, -rbrace, -sequence, mercury, hit, hit, comes, obviously, version, nmh, using, delta, pick, -version, pick, nmh-, compiled, fuchsia, cs, mu, oz, au, sun, mar, ict, relevant, part, mh, profile, delta, mhparam, pick, -seq, sel, ...]",0


,subject,to,from,email_body,label
8595,"[cannabis, difference]",webnote.net,dialix.oz.au,"[mid-summer, customer, appreciation, sale, express, appreciation, loyal, customers, offering, following, products, limited, time, ragga, dagga, stoney, mahoney, sweet, vjestika, aqueous, kathmandu, along, free, oz, package, capillaris, herba, three, new, renegade, botanicals, exotic, botanical, resources, ethnobotanical, herbalists, brought, herba, supplementals, kathmandu, temple, kiff, personal, choice, pipe-smoking, products, substances, common, market, announcing, temple, ragga, dagga, tm, aqueous, kathmandu, tm, stoney, mahoney, tm, temple, ragga, dagga, finally, able, offer, sensitive, responsive, personal, choice, smoking, enjoyment, temple, ragga, dagga, tm, pipe-smoking, substance, supplemental, product, introduced, three, years, research, development, temple, personal, choice, legal, smoking, indulgence, redefined, thanks, recent, dramatic, technological, advances, laboratorial, processes, extraction, ...]",1
8596,"[ilug, wilson, kamela]",linux.ie,netscape.net,"[attn, sir, madan, strictly, confidential, pleased, introduce, name, mr, wilson, kamela, native, south, africa, senior, employee, mines, natural, resources, department, currently, trainning, course, holland, months, writing, letter, request, assistance, order, redeem, investment, south, african, mining, corporation, said, investment, valued, million, dollars, fifteen, million, five, hundred, thousand, dollars, purchased, lucio, harper, contracted, south, african, mining, corporation, recognised, mines, natural, resources, department, redeemable, investment, interest, matured, since, march, last, year, since, march, last, year, several, attempts, made, contact, lucio, harper, without, success, way, contact, close, relatives, whose, favour, investment, cash, value, paid, since, access, lucio, harper, information, claim, money, help, partners, south, ...]",1


In [ ]:
# Handling typos and mispellings
english_vocab = set(w.lower() for w in words.words())


def lemmatize_word(word, pos=None):
    lemmatizer = WordNetLemmatizer()
    if pos is None:
        return lemmatizer.lemmatize(word)        
    
    return lemmatizer.lemmatize(word, pos=pos)


def suggest_words(word, threshold=3):
    suggested_words = [e_word for e_word in english_vocab if edit_distance(word, e_word) <= threshold]
    return suggested_words


def is_combined_word(word_list, index):
    combined_word = word_list[index] + word_list[index+1]
    lemmatized_word = lemmatize_word(combined_word, 'v')

    # Check if the combined word is in the dictionary and not a stopword
    if lemmatized_word in english_vocab and lemmatized_word not in stop_words:
        word_list[index] = lemmatize_word(combined_word)
        word_list.pop(index+1)
        return True

    return False


def clean_words(word_list):
    word_list_size = len(word_list)-1
    idx = 0
    
    while idx < word_list_size:
        # Remove trailing and leading dash(es)
        cleaned_word = re.sub(r'^-+|-+$', '', word_list[idx])

        # Check if any of two consecutive words (current and after) aren't in dictionary and 
        # see if putting them together might work
        if any(word not in english_vocab for word in word_list[idx:idx+2]):
            # Check if two words can be combined
            is_valid_word = is_combined_word(word_list, idx)
            if is_valid_word:
                word_list_size = word_list_size - 1
            else:
                # Remove one-character word from the list
                if len(cleaned_word) == 1:
                    word_list.pop(idx)    
                    word_list_size = word_list_size - 1
                else:
                    # suggested_words = suggest_words(cleaned_word)
                    # TODO: Improve this section of the code
                    # For now, we only take words that have error distance of 3
                    # if len(suggested_words) == 0:
                    #     word_list.pop(idx)    
                    #     word_list_size = word_list_size - 1
                    # else:
                    word_list[idx] = lemmatize_word(cleaned_word)
        else:
            word_list[idx] = lemmatize_word(cleaned_word)
            
        idx = idx + 1

    return word_list


data_cleaned_df[['subject', 'email_body']] = data_cleaned_df[['subject', 'email_body']].applymap(lambda x : clean_words(x))
data_cleaned_df[['subject', 'email_body']]

,subject,email_body
0,"[new, sequence, window]","[date, tue, aug, chris, garrigues, message-id, hoping, people, additional, sequence, notice, purely, cosmetic, change, well, first, exmh, latest, one, change, start, get, read, flist, totalcount, unseen, element, array, executing, totalcount, unseen-sequence, flaginner, spool, iconspool, labelup, else, flaginner, icondown, labeldown, procedure, flag, msgseen, line, invoked, within, flag, msgseen, procedure, msgseen, line, invoked, within, msgseen, procedure, msgshow, line, invoked, within, msgshow, procedure, msgchange, line, invoked, within, msgchange, show, invoked, within, time, list, msgchange, procedure, msg, change, line, invoked, within, msg, change, id, procedure, msg, show, line, invoked, within, msg, show, cur, eval, body, line, invoked, within, eval, procedure, folderchange, line, invoked, within, ...]"
1,"[new, sequence, window]","[date, wed, aug, chris, garrigues, message-id, reproduce, error, repeatable, like, every, time, without, fail, debug, log, pick, happening, pick, exec, pick, inbox, list, lbrace, lbrace, subject, ftp, rbrace, rbrace, sequence, mercury, exec, pick, inbox, list, lbrace, lbrace, subject, ftp, rbrace, rbrace, sequence, mercury, ftoc, pickmsgs, hit, marking, hit, tkerror, syntax, error, expression, int, note, run, pick, command, hand, delta, pick, inbox, list, lbrace, lbrace, subject, ftp, rbrace, rbrace, sequence, mercury, hit, hit, come, obviously, version, nmh, using, delta, pick, version, pick, nmh, compiled, fuchsia, c, mu, oz, au, sun, mar, ict, relevant, part, mh, profile, delta, mhparam, pick, seq, sel, ...]"
2,"[personal, finance, resolution, keep]","[motley, fool, personal, finance, wednesday, january, issue, ask, fool, stop, solicitation, money, resolution, keep, spotlight, saving, market, community, tip, least, one, thing, differently, sponsored, datek, datek, online, built, trade, proprietary, auto, routing, technology, commission, online, equity, trade, second, execution, commitment, apply, datek, account, ask, fool, q, tired, getting, many, credit, card, offer, mail, already, card, want, mail, offering, new, one, stop, three, main, credit, bureau, united, state, agreed, someone, contact, one, asks, removed, junk, mail, er, direct, mail, list, telemarketing, phone, list, honor, request, full, answer, money, resolution, keep, let, u, face, new, year, resolution, mostly, nightmare, hang, head, like, ...]"
3,"[new, sequence, window]","[date, wed, aug, exmh, content-type, text, plain, charset, us-ascii, tue, aug, edt, said, ever, tried, get, mh, pseq, sequence, suspect, everybod, looking, big, box, unseen, pseq, might, want, add, pseq, hide, default, list, intended, added, sequence, never, show, list, take, effect, till, stopped, restarted, exmh, added, pseq, hit, save, preference, take, effect, till, restarted, one, point, worked, fine, check, see, stopped, working, chris, chris, garrigues, cwg, vircio, congress, suite, austin, tx, world, war, iii, wrong-doers, v, evil-doers, begin, pgp, signature, version, gnupg, v, gnu, linux, comment, exmh, version, id, dbqe, gk, b, h, iuirasiyaj, zejm, tiqdd, mqu, lbapzzpasgcccjtl, bdfb, wkmtagwylf, ...]"
4,"[zzzzteana, alexander]","[martin, posted, tasso, papadopoulos, greek, sculptor, behind, plan, judged, limestone, mount, kerdylio, mile, east, salonika, far, mount, athos, monastic, community, ideal, patriotic, sculpture, well, alexander, granite, feature, ft, high, ft, wide, museum, restored, amphitheatre, car, park, admiring, crowd, planned, mountain, limestone, granite, limestone, weather, pretty, fast, yahoo, group, sponsor, dvd, free, p, join, unsubscribe, group, send, email, use, yahoo, group, subject]"
...,...,...
8592,"[busy, home, study, make, sense]","[want, bos, train, self-paced, home, study, thousand, people, people, like, able, work, home, thanks, convenience, distance, learning, course, pcdi, bonus, tuition, discount, enroll, pcdi, offer, popular, career, training, program, inclu

## Building Model (uncleaned data)

Note: Make sure to run 3 iterations of tuning and explain your rationale for the tuning approaches used each iteration

In [ ]:
# Split independent variables and dependent variable
X_raw = data_raw_df.drop(['label'], axis=1)
y_raw = data_raw_df['label']

X_raw.shape, y_raw.shape

((8597, 4), (8597,))

### Model Evaluation

Note: Evaluate both test and train data. Make sure there are enough data points in the test set (>500) for Confusion Matrix, AUC etc.

## Building Model (cleaned data)

Note: Make sure to run 3 iterations of tuning and explain your rationale for the tuning approaches used each iteration

In [ ]:
# Split independent variables and dependent variable
X_cleaned = data_cleaned_df.drop(['label'], axis=1)
y_cleaned = data_cleaned_df['label']

X_cleaned.shape, y_cleaned.shape

((8597, 4), (8597,))

### Model Evaluation

Note: Evaludate both test and train data. Make sure there are enough data points in the test set (>500) for Confusion Matrix, AUC etc.

## Conclusion

Note: Include what else could be done to tune the model and how it would have helped (w/ some numbers)

## References
- https://coderzcolumn.com/tutorials/python/email-how-to-represent-an-email-message-in-python